In [120]:
from palmerpenguins import load_penguins
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn import FunctionSampler
from sklearn.base import BaseEstimator, TransformerMixin
import scipy
import pandas as pd

from collections import defaultdict
d = defaultdict(LabelEncoder)

In [59]:
df = load_penguins()
# Find unbalanced class
Counter(df["island"])
# Count missing values
df.isnull().sum(axis=1)
#OnehotEncode categorical values
categorical_transformer = Pipeline(steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))])
cate = categorical_transformer.fit(df)

# print(new_df)
# Outlier values
upper_limit = df["bill_length_mm"].mean() + 3 * df["bill_length_mm"].std()
lower_limit = df["bill_length_mm"].mean() - 3 * df["bill_length_mm"].std()
print(upper_limit, lower_limit)

60.300680966341 27.543178682781807


In [41]:
Counter(df["island"])

Counter({'Biscoe': 168, 'Dream': 124, 'Torgersen': 52})

In [14]:
r = df.isnull().sum(axis=1)
np.array(r)

array([0, 0, 0, 5, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Extract the data and target from the DataFrame

In [132]:
X_cate = df.loc[:, ['island','sex','year']]
X_num = df.loc[:, 'bill_length_mm':'body_mass_g']
y_data = df.loc[:, ['species']]

In [112]:

class MyCustomTransformer(BaseEstimator):
    def __init__(self, hp=None):
        self.hp = hp
        self.row_null = None
    def fit(self, X, y=None):
        try:
            # Get the categoritics with too many missing values by row
            num_of_null = X.isnull().sum(axis=1)
            self.row_null = list(num_of_null[num_of_null/len(X.columns) >= 0.1].index)
        except Exception as e:
            print(e)
        finally:
            return self

    def transform(self, X, y=None):
        try:
            # Remove the categoritics with too many missing values
            X_tmp = X.drop(self.row_null)
            y_tmp = y.drop(self.row_null)
            # Fill the missing value with the categoritics have fewer missing values
            imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
            imp.fit(X_tmp)

            # Use OneHotEncoder to the categorical data
            encoder = OneHotEncoder(dtype='int', handle_unknown='ignore')
            X_raw = encoder.fit_transform(X_tmp.loc[:, ['island','sex','year']])
            y_raw = encoder.fit_transform(y_tmp)
        except Exception as e:
            print(e)
        finally:
            return X_raw, y_raw

In [118]:
custom_transform = MyCustomTransformer()
custom_transform.fit(X_cate, y_data)
X_cate_new, y_new = custom_transform.transform(X_cate, y_data)
a = y_new.toarray()

In [135]:

z = np.abs(scipy.stats.zscore(X_num, nan_policy='omit'))
a = X_num[z<=1]
c = a[~a.isnull().any(axis=1)]

In [ ]:
class MyCustomSampler(BaseEstimator):
    def __init__(self, strategy):
        self.strategy = strategy

    def fit_resample(self, X, y=None):
        if(self.strategy == 'z_score'):
            z = np.abs(scipy.stats.zscore(X, nan_policy='omit'))
            X_tmp = X[z <= 3 and z >= -3]
        elif(self.strategy == 'IQR'):
            
        else:
            print("Invalid strategy")
        return X, y
